In [2]:
from tensorflow.keras.optimizers import Adam
from keras.utils import Sequence
import pandas as pd
import numpy as np
import os

pi = 3.14159265359

maxval=1e9
minval=1e-9

2025-03-29 12:20:31.310035: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743268831.332401 3941807 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743268831.338773 3941807 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743268831.355434 3941807 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743268831.355456 3941807 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743268831.355460 3941807 computation_placer.cc:177] computation placer alr

In [3]:
from dataloaders.OptimizedDataGenerator import OptimizedDataGenerator
from models.models import *

ModuleNotFoundError: No module named 'qkeras'

In [3]:
model=CreateModel_v2((13,21,20))
model.summary()

2025-03-08 19:06:34.725515: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3234 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 1g.5gb, pci bus id: 0000:c1:00.0, compute capability: 8.0
/usr/local/lib/python3.8/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13, 21, 20)]      0         
                                                                 
 q_separable_conv2d (QSepar  (None, 9, 17, 32)         1172      
 ableConv2D)                                                     
                                                                 
 q_activation (QActivation)  (None, 9, 17, 32)         0         
                                                                 
 q_separable_conv2d_1 (QSep  (None, 5, 13, 16)         1328      
 arableConv2D)                                                   
                                                                 
 q_activation_1 (QActivatio  (None, 5, 13, 16)         0         
 n)                                                              
                                                             

In [16]:
# get best weights file
pitch = '50x12P5'
batch_size = 5000
fingerprint = '1818e811'
files = os.listdir('/home/dajiang/smart-pixels-ml/weights/weights_7pitches/dataset_3sr_cotBeta1P5_haarTransformed_weights/weights-{}-bs{}-{}-checkpoints'.format(pitch, batch_size, fingerprint))

vlosses = [float(f.split("-v")[1].split(".hdf5")[0]) for f in files]
bestfile = files[np.argmin(vlosses)]
#model.load_weights('/home/dajiang/smart-pixels-ml/weights/weights_7pitches/dataset_3sr_cotBeta1P5_haarTransformed_weights/weights-{}-bs{}-{}-checkpoints/'.format(pitch, batch_size, fingerprint)+bestfile)

In [17]:
bestfile

'weights.386-t-20552.54-v-22209.09.hdf5'

In [6]:
# load in the test set
test_generator = OptimizedDataGenerator(
    load_from_tfrecords_dir = '/data/dajiang/smart-pixels/tfrecords/tfrecords_dataset_3sr_{}_20t_bs5000_cotBeta1P5_val'.format(pitch),
    quantize = True
)

In [7]:
# predicts test data
p_test = model.predict(test_generator)

complete_truth = None
for _, y in test_generator:
    if complete_truth is None:
        complete_truth = y
    else:
        complete_truth = np.concatenate((complete_truth, y), axis=0)

# creates df with all predicted values and matrix elements - 4 predictions, all 10 unique matrix elements
df = pd.DataFrame(p_test,columns=['x','M11','y','M22','cotA','M33','cotB','M44','M21','M31','M32','M41','M42','M43'])

# stores all true values in same matrix as xtrue, ytrue, etc.
df['xtrue'] = complete_truth[:,0]
df['ytrue'] = complete_truth[:,1]
df['cotAtrue'] = complete_truth[:,2]
df['cotBtrue'] = complete_truth[:,3]
df['M11'] = minval+tf.math.maximum(df['M11'], 0)
df['M22'] = minval+tf.math.maximum(df['M22'], 0)
df['M33'] = minval+tf.math.maximum(df['M33'], 0)
df['M44'] = minval+tf.math.maximum(df['M44'], 0)

df['sigmax'] = abs(df['M11'])
df['sigmay'] = np.sqrt(df['M21']**2 + df['M22']**2)
df['sigmacotA'] = np.sqrt(df['M31']**2+df['M32']**2+df['M33']**2)
df['sigmacotB'] = np.sqrt(df['M41']**2+df['M42']**2+df['M43']**2+df['M44']**2)

# calculates residuals for x, y, cotA, cotB
df['residualsX'] = df['xtrue'] - df['x']
df['residualsY'] = df['ytrue'] - df['y']
df['residualsA'] = df['cotAtrue'] - df['cotA']
df['residualsB'] = df['cotBtrue'] - df['cotB']

# calculates pulls for x, y, cotA, cotB
df['pullx'] = (df['xtrue']-df['x'])/df['sigmax']
df['pully'] = (df['ytrue']-df['y'])/df['sigmay']
df['pullcotA'] = (df['cotAtrue']-df['cotA'])/df['sigmacotA']
df['pullcotB'] = (df['cotBtrue']-df['cotB'])/df['sigmacotB']

# stores results as parquet
df.to_parquet("/home/dajiang/smart-pixels-ml/processed_parquets/dataset_3sr_cotBeta1P5/{}_bs{}_{}_20t_dataset3sr_v2_vars.parquet".format(pitch, batch_size, date))

 2/45 [>.............................] - ETA: 4s 

2025-03-08 19:06:57.421001: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-03-08 19:06:57.457518: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8906


45/45 [==============================] - 7s 149ms/step
